# Cellular Sampler
The following code generates the transition matrices and stationary distributions for the cellular sampler

In [2]:
import numpy as np
from numpy.linalg import eig, qr, solve
from scipy.signal import convolve2d

import matplotlib.pyplot as plt

In [3]:
n = 10
k = 3


def create_transition_matrix(n,k,algorithm,projection,c=None):
    if algorithm == "grid-walk":
        T = np.zeros((n,n))
        w=2*k+1
        p=1/w    
        for i in range(k+1):
            np.fill_diagonal(T[i:,:],p)
            np.fill_diagonal(T[:,i:],p)
        if projection == True:
            for i in range(k+1):
                T[0,i] = (k+1-i) * p
                T[-1,-i-1] = (k+1-i) * p
            T = T.T
        else:
            for i in range(k+1):
                T[:,i] /= p
                T[:,i] *= (1 / (w-k+i))
                T[:,-i-1] /= p
                T[:,-i-1] *= 1 / (w-k+i)
            T = T.T
    elif algorithm == "RRT":
        w = n+2*c
        T = np.zeros((w,w))
        if projection == True:
            p = 1/w
            T[:,c:-c] = p     
            p_border = c*p + 0.5*p*k + 0.5*p
            T[:,c] = p_border
            T[:,-c-1] = p_border
            for i in range(1,n-1):
                if i < k or i+k >= n:
                    T[:,c+i] = p*0.5 
                else:
                    T[:,c+i] = p
        else:
            p = 1/(n-k)
            T[:,c:-c] = p     
            p_border = 0.5*p
            T[:,c] = p_border
            T[:,-c-1] = p_border
            for i in range(1,n-1):
                if i < k or i+k >= n:
                    T[:,c+i] = p*0.5 
                else:
                    T[:,c+i] = p
        T = T[c:c+n,c:c+n]
    else:
        print("choose algorithm = 'RRT' | 'grid-walk'")
    return T


def solve_stationary_distribution(T):
    n = len(T)
    I = np.identity(n)
    T_new = T.T-I
    A = np.zeros((n+1,n))
    A[:-1,:] = T_new
    A[-1,:] = 1
    Q,R = qr(A,mode="reduced")
    b = np.zeros(n+1)
    b[-1] = 1
    Qb = np.matmul(Q.T,b)
    x_qr = solve(R,Qb)
    return x_qr

def evolve_chain(T,n_iter, x_0=None):
    n = len(T)
    x_i = np.zeros((n_iter,n))
    if x_0 is None:
        x = np.ones(n)/n
    else:
        x = x_0
    x_i[0,:] = x
    for i in range(1,n_iter):
        x = x@T
        x_i[i,:] = x.copy()
    return x_i

In [3]:
# grid walk
# number of cells
n = 100
# neighborhood
ks = [10,20,30]

for k in ks:
    T = create_transition_matrix(n,k,"grid-walk",True,c=None)
    x_qr = solve_stationary_distribution(T)
    np.savetxt(f'grid_walk_projection_discrete_transitions_{k}.out', T) 
    np.savetxt(f'grid_walk_projection_discrete_stationary_{k}.out', x_qr) 
    
for k in ks:
    T = create_transition_matrix(n,k,"grid-walk",False,c=None)
    x_qr = solve_stationary_distribution(T)
    np.savetxt(f'grid_walk_rejection_discrete_transitions_{k}.out', T) 
    np.savetxt(f'grid_walk_rejection_discrete_stationary_{k}.out', x_qr)     

In [4]:
# RRT
# number of cells
n = 100
# neighborhood
k = 1
cs = [30,50,70]

for c in cs:
    T = create_transition_matrix(n,k,"RRT",True,c=c)
    x_qr = solve_stationary_distribution(T)
    np.savetxt(f'rrt_projection_discrete_transitions_{c}.out', T) 
    np.savetxt(f'rrt_projection_discrete_stationary_{c}.out', x_qr) 
    
for c in cs:
    T = create_transition_matrix(n,k,"RRT",False,c=c)
    x_qr = solve_stationary_distribution(T)
    np.savetxt(f'rrt_rejection_discrete_transitions_{c}.out', T) 
    np.savetxt(f'rrt_rejection_discrete_stationary_{c}.out', x_qr)     

In [4]:
def create_transition_matrix_circle(n,k):
    T = np.zeros((n,n))
    p = 1/(2*k+1)
    for j in range(k):
        T[j,0:k+1+j] = p
        T[j,-k+j:] = p
        T[-j-1,0:k-j] = p
    for i in range(n):
        T[i,i-k:i+k+1] = p
    return T

T = create_transition_matrix_circle(10,2)
x_qr = solve_stationary_distribution(T)
print(x_qr)

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


In [5]:
T

array([[0.2, 0.2, 0.2, 0. , 0. , 0. , 0. , 0. , 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. , 0. , 0. , 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.2, 0.2, 0.2, 0.2, 0.2, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0. , 0. , 0. , 0. , 0. , 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0. , 0. , 0. , 0. , 0. , 0.2, 0.2, 0.2]])

In [6]:
np.savetxt("circle_transitions.out",T)
np.savetxt("circle_stationary.out", x_qr)